# Plotting DVL and Slocum Glider Data with Pandas & Seaborn

---
## Import Libraries

In [1]:
import datetime
import earthpy as et
import earthpy.plot as ep
import importlib
import numpy as np
import os
import pandas as pd
import rasterio as rio
import scipy
import seaborn as sns 
import struct
import sys
import utm
import unittest
from PIL import Image
from matplotlib import pyplot as plt 
from os import listdir
from os.path import isfile, join

# add parent directory to the path for importing modules 
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import PathfinderDVL
import PathfinderEnsemble
import PathfinderTimeSeries
import PathfinderUtils
import SlocumFlightController
import SlocumScienceController
import dvl_plotter
sns.set()

def reload_modules():
    importlib.reload(PathfinderDVL)
    importlib.reload(PathfinderEnsemble)
    importlib.reload(PathfinderTimeSeries)
    importlib.reload(PathfinderUtils)
    importlib.reload(SlocumFlightController)
    importlib.reload(SlocumScienceController)
    importlib.reload(dvl_plotter)

---
## Load or Parse DVL data 

In [ ]:
reload_modules()
glider = "sentinel"
filepath = "/Users/zduguid/Dropbox (MIT)/MIT-WHOI/Kolumbo cruise 2019/zduguid/pd0-raw/%s/" % (glider)
# filename = "01820002.pd0" # example of prime data set to work with
# filename = "01740000.pd0" # example of file not parsing correctly
filename = "sk261222.pd0" # example of another cool mission (less bottom track, long dive)
ts = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename, save=False)

In [ ]:
# filter out bin0, bin1 -- something fishy going on here
# pursue bin2+
pitch_bias = 12
bin_len = ts2.df.depth_bin_length[0]
bin0_dist = ts2.df.bin0_distance[0]

v_oc_abs_x = ts2.df.abs_vel_btm_u - ts2.df.rel_vel_dvl_u
v_oc_abs_y = ts2.df.abs_vel_btm_v - ts2.df.rel_vel_dvl_v

v_bin2_to_bin3_x = -ts2.df.vel_bin2_beam0 + ts2.df.vel_bin3_beam0
v_bin2_to_bin3_y = -ts2.df.vel_bin2_beam1 + ts2.df.vel_bin3_beam1

v_pres_to_bin2_x = ts2.df.rel_vel_pressure_u + ts2.df.vel_bin2_beam0
v_pres_to_bin2_y = ts2.df.rel_vel_pressure_v + ts2.df.vel_bin2_beam1






fig, ax = plt.subplots(figsize=(15,10))
# sns.scatterplot(v_bin2_to_bin3_y, -ts2.df.depth, label=r"y")
# sns.scatterplot(v_pres_to_bin2_x, -ts2.df.depth, label=r"x")
# sns.scatterplot(v_pres_to_bin2_y, -ts2.df.depth, label=r"y")
sns.scatterplot(v_bin2_to_bin3_x, -ts2.df.depth, label=r"$-v_{DVL_2}-(-v_{DVL_3})$")
sns.scatterplot(v_pres_to_bin2_x, -ts2.df.depth, label=r"$v_{DR}-(-v_{DVL})$")
sns.scatterplot(v_oc_abs_x, -ts2.df.depth, label=r"$v_{OG}-v_{tw}$")
# sns.scatterplot(v_oc_abs_y, -ts2.df.depth, label=r"$v_{oc}$")

# plt.ylim(-1,1)


In [3]:
sns.distplot(ts.df.delta_z)

NameError: name 'ts' is not defined

In [57]:
reload_modules()
import unittest

class TestOceanCurrentPropagation(unittest.TestCase):
    """Test shear-based ocean current propgation methods."""
    
    def test_bad_ocean_current_format(self):
        with self.assertRaises(ValueError):
            PathfinderUtils.OceanCurrent(0,1,None) 
            
    def test_one_observation(self):
        # set up the problem
        z = 0
        t = 0
        water_column = PathfinderUtils.WaterColumn(max_depth=20)
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0),(2,0,0),(3,0,0)]]
        voc_ref = PathfinderUtils.OceanCurrent(0,0,0)
        water_column.add_shear_node(z,t,shear_list,voc_ref)
        voc0 = water_column.get_voc_at_depth(0)[0]
        voc2 = water_column.get_voc_at_depth(2)[0]
        voc4 = water_column.get_voc_at_depth(4)[0]
        voc6 = water_column.get_voc_at_depth(6)[0]
        
        # test that ShearNodes have the correct flags set
        self.assertTrue(voc0.btm_track)
        self.assertTrue(voc2.fwd_prop)
        self.assertTrue(voc4.fwd_prop)
        self.assertTrue(voc6.fwd_prop)
        
        # test that ocean current velocities are correct
        self.assertEqual(voc0.voc, PathfinderUtils.OceanCurrent( 0,0,0))
        self.assertEqual(voc2.voc, PathfinderUtils.OceanCurrent(-1,0,0))
        self.assertEqual(voc4.voc, PathfinderUtils.OceanCurrent(-2,0,0))
        self.assertEqual(voc6.voc, PathfinderUtils.OceanCurrent(-3,0,0))
        
        
    def test_forward_propagation_descending(self):
        # observation 1 
        z = 0
        t = 0
        water_column = PathfinderUtils.WaterColumn(max_depth=20)
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0),(2,0,0),(3,0,0)]]
        voc_ref = PathfinderUtils.OceanCurrent(0,0,0)
        water_column.add_shear_node(z,t,shear_list,voc_ref)
        # observation 2
        z = 2
        t = 1
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(0,0,0),(1,0,0)]]
        voc_ref = PathfinderUtils.OceanCurrent()
        water_column.add_shear_node(z,t,shear_list,voc_ref)
        # get shear node lists 
        voc0 = [str(sn.voc) for sn in water_column.get_voc_at_depth(0)]
        voc2 = [str(sn.voc) for sn in water_column.get_voc_at_depth(2)]
        voc4 = [str(sn.voc) for sn in water_column.get_voc_at_depth(4)]
        voc6 = [str(sn.voc) for sn in water_column.get_voc_at_depth(6)]
        # check that lengths correct 
        self.assertEqual(len(voc0), 1)
        self.assertEqual(len(voc2), 1)
        self.assertEqual(len(voc4), 2)
        self.assertEqual(len(voc6), 2)
        # check that correct velocities present 
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0,0,0)) in voc0)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-1,0,0)) in voc2)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-1,0,0)) in voc4)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-2,0,0)) in voc4)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-2,0,0)) in voc6)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-3,0,0)) in voc6)
    
    def test_forward_propagation_ascending(self):
        water_column = PathfinderUtils.WaterColumn(max_depth=12)
        # observation 1
        z = 8
        t = 0
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0)]]
        direction='ascending'
        voc_ref = PathfinderUtils.OceanCurrent(5,0,0)
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # observation 2
        z = 4
        t = 2
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(-1,0,0),(0,0,0),(1,0,0)]]
        direction='ascending'
        voc_ref = PathfinderUtils.OceanCurrent()
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=voc_ref)
        # get shear node lists 
        voc4  = [str(sn.voc) for sn in water_column.get_voc_at_depth(4)]
        voc6  = [str(sn.voc) for sn in water_column.get_voc_at_depth(6)]
        voc8  = [str(sn.voc) for sn in water_column.get_voc_at_depth(8)]
        voc10 = [str(sn.voc) for sn in water_column.get_voc_at_depth(10)]
        # check that lengths correct 
        self.assertEqual(len(voc4),  1)
        self.assertEqual(len(voc6),  1)
        self.assertEqual(len(voc8),  1)
        self.assertEqual(len(voc10), 2)
        # check that correct velocities present 
        self.assertTrue(str(PathfinderUtils.OceanCurrent(5,0,0)) in voc4)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(6,0,0)) in voc6)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(5,0,0)) in voc8)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(4,0,0)) in voc10)
        
    def test_back_propagation_descending(self):
        water_column = PathfinderUtils.WaterColumn(max_depth=14)
        # observation 1
        z = 2
        t = 1
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(.2,.4,-.2),(.2,.4,-.2),(.4,.8,-.4)]]
        direction='descending'
        voc_ref = PathfinderUtils.OceanCurrent()
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # observation 2
        z = 6
        t = 2
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(.1,.2,-.1),(.2,.4,-.2)]]
        direction='descending'
        voc_ref = PathfinderUtils.OceanCurrent()
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # observation 3
        z = 10
        t = 3
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(0,0,0)]]
        direction='descending'
        voc_ref = PathfinderUtils.OceanCurrent(0,0,0)
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # observation 4
        z = 8
        t = 4
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,2,-1)]]
        direction='ascending'
        voc_ref = PathfinderUtils.OceanCurrent()
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # get shear node lists 
        voc2  = [str(sn.voc) for sn in water_column.get_voc_at_depth(2)]
        voc4  = [str(sn.voc) for sn in water_column.get_voc_at_depth(4)]
        voc6  = [str(sn.voc) for sn in water_column.get_voc_at_depth(6)]
        voc8  = [str(sn.voc) for sn in water_column.get_voc_at_depth(8)]
        voc10 = [str(sn.voc) for sn in water_column.get_voc_at_depth(10)]
        voc12 = [str(sn.voc) for sn in water_column.get_voc_at_depth(12)]
        # check that lengths correct 
        self.assertEqual(len(voc2),  1)
        self.assertEqual(len(voc4),  1)
        self.assertEqual(len(voc6),  1)
        self.assertEqual(len(voc8),  3)
        self.assertEqual(len(voc10), 1)
        self.assertEqual(len(voc12), 1)
        # check that correct velocities present 
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0.4,0.8,-0.4)) in voc2)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0.2,0.4,-0.2)) in voc4)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0.2,0.4,-0.2)) in voc6)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0,0,0))        in voc8)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0.1,0.2,-0.1)) in voc8)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(1,2,-1))       in voc8)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0,0,0))        in voc10)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0,0,0))        in voc12)
        
        
    def test_back_propagation_ascending(self):
        water_column = PathfinderUtils.WaterColumn(max_depth=14)
        # observation 1
        z = 10
        t = 1
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(2,0,0)]]
        direction='ascending'
        voc_ref = PathfinderUtils.OceanCurrent()
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # observation 2
        z = 8
        t = 2
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0)]]
        direction='ascending'
        voc_ref = PathfinderUtils.OceanCurrent()
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # observation 3
        z = 2
        t = 3
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(-1,0,0),(0,0,0),(1,0,0)]]
        direction='ascending'
        voc_ref = PathfinderUtils.OceanCurrent(1,0,0)
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # get shear node lists 
        voc2  = [str(sn.voc) for sn in water_column.get_voc_at_depth(2)]
        voc4  = [str(sn.voc) for sn in water_column.get_voc_at_depth(4)]
        voc6  = [str(sn.voc) for sn in water_column.get_voc_at_depth(6)]
        voc8  = [str(sn.voc) for sn in water_column.get_voc_at_depth(8)]
        voc10 = [str(sn.voc) for sn in water_column.get_voc_at_depth(10)]
        voc12 = [str(sn.voc) for sn in water_column.get_voc_at_depth(12)]
        # check that lengths correct 
        self.assertEqual(len(voc2),  1)
        self.assertEqual(len(voc4),  1)
        self.assertEqual(len(voc6),  1)
        self.assertEqual(len(voc8),  1)
        self.assertEqual(len(voc10), 1)
        self.assertEqual(len(voc12), 1)
        # check that correct velocities present 
        self.assertTrue(str(PathfinderUtils.OceanCurrent(1, 0,0)) in voc2)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(2, 0,0)) in voc4)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(1, 0,0)) in voc6)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0, 0,0)) in voc8)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-1,0,0)) in voc10)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-3,0,0)) in voc12)
    
    def test_bin_filter(self):
        water_column = PathfinderUtils.WaterColumn(max_depth=14,bin_filter=2)
        # observation 1
        z = 0
        t = 1
        shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0),(2,0,0),(3,0,0),(4,0,0)]]
        direction='descending'
        voc_ref = PathfinderUtils.OceanCurrent(0,0,0)
        water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
        # get shear node lists 
        voc0  = [str(sn.voc) for sn in water_column.get_voc_at_depth(0)]
        voc2  = [str(sn.voc) for sn in water_column.get_voc_at_depth(2)]
        voc4  = [str(sn.voc) for sn in water_column.get_voc_at_depth(4)]
        voc6  = [str(sn.voc) for sn in water_column.get_voc_at_depth(6)]
        voc8  = [str(sn.voc) for sn in water_column.get_voc_at_depth(8)]
        # check that lengths correct 
        self.assertEqual(len(voc0),  1)
        self.assertEqual(len(voc2),  0)
        self.assertEqual(len(voc4),  0)
        self.assertEqual(len(voc6),  1)
        self.assertEqual(len(voc8),  1)
        # check that correct velocities present 
        self.assertTrue(str(PathfinderUtils.OceanCurrent(0, 0,0)) in voc0)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-3,0,0)) in voc6)
        self.assertTrue(str(PathfinderUtils.OceanCurrent(-4,0,0)) in voc8)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored','-v'], exit=False)

test_back_propagation_ascending (__main__.TestOceanCurrentPropagation) ... ok
test_back_propagation_descending (__main__.TestOceanCurrentPropagation) ... ok
test_bad_ocean_current_format (__main__.TestOceanCurrentPropagation) ... ok
test_bin_filter (__main__.TestOceanCurrentPropagation) ... ok
test_forward_propagation_ascending (__main__.TestOceanCurrentPropagation) ... ok
test_forward_propagation_descending (__main__.TestOceanCurrentPropagation) ... ok
test_one_observation (__main__.TestOceanCurrentPropagation) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.017s

OK


In [56]:
water_column = PathfinderUtils.WaterColumn(max_depth=14,bin_filter=2)

# observation 1
z = 0
t = 1
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0),(2,0,0),(3,0,0),(4,0,0)]]
direction='descending'
voc_ref = PathfinderUtils.OceanCurrent(0,0,0)
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)
print(water_column)

Water Column (depth=14) 
|z =  0| Shear<z:  0, t:   1, V[   0,   0,   0], btm-trck>
|z =  2|
|z =  4|
|z =  6| Shear<z:  6, t:   1, V[-300,   0,   0], fwd-prop>
|z =  8| Shear<z:  8, t:   1, V[-400,   0,   0], fwd-prop>
|z = 10|
|z = 12|



In [43]:
#######################################################
# MODE 4: Ascending Back Propagation ##################
#######################################################
reload_modules()
water_column = PathfinderUtils.WaterColumn(max_depth=14)

# observation 1
z = 10
t = 1
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(2,0,0)]]
direction='ascending'
voc_ref = PathfinderUtils.OceanCurrent()
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

# observation 2
z = 8
t = 2
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0)]]
direction='ascending'
voc_ref = PathfinderUtils.OceanCurrent()
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

# observation 3
z = 2
t = 3
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(-1,0,0),(0,0,0),(1,0,0)]]
direction='ascending'
voc_ref = PathfinderUtils.OceanCurrent(1,0,0)
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

print(water_column)

Water Column (depth=14) 
|z =  0|
|z =  2| Shear<z:  2, t:   3, V[ 100,   0,   0], btm-trck>
|z =  4| Shear<z:  4, t:   3, V[ 200,   0,   0], fwd-prop>
|z =  6| Shear<z:  6, t:   3, V[ 100,   0,   0], fwd-prop>
|z =  8| Shear<z:  8, t:   3, V[   0,   0,   0], fwd-prop>
|z = 10| Shear<z: 10, t:   1, V[-100,   0,   0], bck-prop>
|z = 12| Shear<z: 12, t:   1, V[-300,   0,   0], bck-prop>



In [49]:
#######################################################
# MODE 3: Descending Back Propagation #################
#######################################################
reload_modules()
water_column = PathfinderUtils.WaterColumn(max_depth=14)

# observation 1
z = 2
t = 1
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(.2,.4,-.2),(.2,.4,-.2),(.4,.8,-.4)]]
direction='descending'
voc_ref = PathfinderUtils.OceanCurrent()
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

# observation 2
z = 6
t = 2
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(.1,.2,-.1),(.2,.4,-.2)]]
direction='descending'
voc_ref = PathfinderUtils.OceanCurrent()
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

# observation 3
z = 10
t = 3
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(0,0,0)]]
direction='descending'
voc_ref = PathfinderUtils.OceanCurrent(0,0,0)
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

# observation 4
z = 8
t = 4
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,2,-1)]]
direction='ascending'
voc_ref = PathfinderUtils.OceanCurrent()
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

print(water_column)

Water Column (depth=14) 
|z =  0|
|z =  2| Shear<z:  2, t:   1, V[  40,  80, -40], bck-prop>
|z =  4| Shear<z:  4, t:   1, V[  20,  40, -20], bck-prop>
|z =  6| Shear<z:  6, t:   1, V[  20,  40, -20], bck-prop>
|z =  8| Shear<z:  8, t:   1, V[   0,   0,   0], bck-prop> Shear<z:  8, t:   2, V[  10,  20, -10], bck-prop> Shear<z:  8, t:   4, V[ 100, 200,-100], fwd-prop>
|z = 10| Shear<z: 10, t:   3, V[   0,   0,   0], btm-trck>
|z = 12| Shear<z: 12, t:   3, V[   0,   0,   0], fwd-prop>



In [48]:
#######################################################
# MODE 2: Ascending Forward Propagation ###############
#######################################################
reload_modules()
water_column = PathfinderUtils.WaterColumn(max_depth=12)

# observation 1
z = 8
t = 0
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0)]]
direction='ascending'
voc_ref = PathfinderUtils.OceanCurrent(5,0,0)
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=direction)

# observation 2
z = 4
t = 2
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(-1,0,0),(0,0,0),(1,0,0)]]
direction='ascending'
voc_ref = PathfinderUtils.OceanCurrent()
water_column.add_shear_node(z,t,shear_list,voc_ref,direction=voc_ref)

print(water_column)

Water Column (depth=12) 
|z =  0|
|z =  2|
|z =  4| Shear<z:  4, t:   2, V[ 500,   0,   0], fwd-prop>
|z =  6| Shear<z:  6, t:   2, V[ 600,   0,   0], fwd-prop>
|z =  8| Shear<z:  8, t:   0, V[ 500,   0,   0], btm-trck>
|z = 10| Shear<z: 10, t:   0, V[ 400,   0,   0], fwd-prop> Shear<z: 10, t:   2, V[ 400,   0,   0], fwd-prop>



In [5]:
#######################################################
# MODE 1: Descending Forward Propagation ##############
#######################################################
reload_modules()
water_column = PathfinderUtils.WaterColumn(max_depth=10)

# observation 1
z = 0
t = 0
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(1,0,0),(2,0,0),(3,0,0)]]
voc_ref = PathfinderUtils.OceanCurrent(0,0,0)
water_column.add_shear_node(z,t,shear_list,voc_ref)

# observation 2
z = 2
t = 1
shear_list = [PathfinderUtils.OceanCurrent(u,v,w) for (u,v,w) in [(0,0,0),(1,0,0)]]
voc_ref = PathfinderUtils.OceanCurrent()
water_column.add_shear_node(z,t,shear_list,voc_ref)

print(water_column)

Water Column (depth=10) 
|z =  0| Shear<z:  0, t:   0, V[   0,   0,   0], btm-trck>
|z =  2| Shear<z:  2, t:   0, V[-100,   0,   0], fwd-prop>
|z =  4| Shear<z:  4, t:   0, V[-200,   0,   0], fwd-prop> Shear<z:  4, t:   1, V[-100,   0,   0], fwd-prop>
|z =  6| Shear<z:  6, t:   0, V[-300,   0,   0], fwd-prop> Shear<z:  6, t:   1, V[-200,   0,   0], fwd-prop>
|z =  8|



In [ ]:
reload_modules()

water_column = PathfinderUtils.WaterColumn()
# print(type(water_column))
print(water_column)

water_column = np.zeros(1000)
pitch_bias   = 8
bin_len      = ts.df.depth_bin_length[0]
bin0_dist    = ts.df.bin0_distance[0]
x_beam = 0
y_beam = 1

bin_len  = np.cos(pitch_bias*np.pi/180)*bin_len
bin0_dist = np.cos(pitch_bias*np.pi/180)*bin0_dist
print(bin_len, bin0_dist)


# for t in range(len(ts.df)):
for t in range(10):
    if not np.isnan(ts.df.abs_vel_btm_u[t]) and t!=0:
        print('bottom lock', i, ts.df.abs_vel_btm_u[t])
        
    else:
        # take difference between dead-reckoned velocity and DVL velocity 
        num_good_vel_bins = ts.df.num_good_vel_bins[t]
        if num_good_vel_bins > 2:
            DR_x  = ts.df.rel_vel_pressure_u[t]
            DR_y  = ts.df.rel_vel_pressure_v[t]
            depth = ts.df.depth[t]
#             print(depth)
            delta_oc_x_list = []
            delta_oc_y_list = []
            delta_z_list = []
            
            # skip the first two bins because they have spurious readings
            for bin_num in range(2, int(num_good_vel_bins)):
                
                x_var = ts.get_profile_var_name('velocity', bin_num, x_beam)
                y_var = ts.get_profile_var_name('velocity', bin_num, y_beam)
                dvl_x = ts.df[x_var][t]
                dvl_y = ts.df[y_var][t]
                
                delta_oc_x = DR_x - (-dvl_x)
                delta_oc_y = DR_y - (-dvl_y)
                delta_z    = bin0_dist + bin_len*bin_num
#                 print(ts.df[x_var][t], ts.df[y_var][t])
#         print(df.num_good_vel_bins[i])

# water_column

In [ ]:
dvl_plotter.plot_profile(ts, glider='sentinel')

In [ ]:
sns.distplot(v_pres_to_bin2_x)
sns.distplot(v_pres_to_bin2_y)

In [ ]:
bin0_dist

In [ ]:
fig = plt.figure(figsize=(15,15))
ax  = fig.gca(projection='3d')

u = v_oc_abs_x[v_oc_abs_x.notnull()]
v = v_oc_abs_y[v_oc_abs_x.notnull()]

vel_max   = 0.3
vel_min   = -vel_max
u[u>vel_max] = np.NaN
v[v>vel_max] = np.NaN
u[u<vel_min] = np.NaN
v[v<vel_min] = np.NaN
z = ts2.df.depth[v_oc_abs_x.notnull()]
t = ts2.df.time[v_oc_abs_x.notnull()]
x = np.zeros(u.shape)
y = np.zeros(u.shape)
w = np.zeros(u.shape)


c = (np.arctan2(u,v) + np.pi)/(2*np.pi)
c = np.concatenate((c, np.repeat(c, 2)))
c = plt.cm.twilight_shifted(c)                        # Colormap
# print(c)
ax.quiver(x, y, -z, u, v, w, colors=c,length=1,normalize=False)
plt.xlabel('Eastward [m/s]')
plt.ylabel('Northward [m/s]')
# ax.elev = 10000
# ax.azim = -90
ax.azim = -45

ax.elev = 30
# ax.elev = 0
ax_max  = 0.3
plt.xlim(-ax_max,ax_max)
plt.ylim(-ax_max,ax_max)
# ax.elev and ax.azim

In [ ]:
fig = plt.figure(figsize=(8,8))
c = np.arctan2(u,v)*180/np.pi

sns.scatterplot(u,v,c,palette='twilight_shifted',legend=False)
plt.axis('equal')
plt.title('Bottom-Lock Ocean Currents')
plt.xlabel('Eastward [m/s]')
plt.ylabel('Northward [m/s]')
# sns.scatterplot(t,v)

In [ ]:
sns.distplot(np.arctan2(v,u).to_numpy())
print(np.max(np.arctan2(v,u)))
print(np.min(np.arctan2(v,u)))

In [ ]:
np.arctan2(-0.1,0)*180/np.pi

In [ ]:
len(v_oc_abs_x)
v_oc_abs_x.dropna()


In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

h1 = ts2.df['btm_beam0_range']
h2 = ts2.df['btm_beam1_range']
h3 = ts2.df['btm_beam2_range']
h4 = ts2.df['btm_beam3_range']
altitude =  ((h1*h2)/(h1 + h2) + (h3*h4)/(h3 + h4))

sns.distplot(ts2.df.num_good_vel_bins)
print(np.min(ts2.df.num_good_vel_bins))
print(np.max(ts2.df.num_good_vel_bins))
set(ts2.df.num_good_vel_bins)

In [ ]:
# fig, ax = plt.subplots(figsize=(15,8))
# sns.scatterplot(ts2.df.num_good_vel_bins, -ts2.df.depth)
# np.mean(ts2.df.num_good_vel_bins)

In [ ]:
reload_modules()
dvl_plotter.plot_velocity_northward(ts2, 'sentinel', roll_size=10)

In [ ]:
ocn_u = ts2.df.abs_vel_btm_u - ts2.df.rel_vel_dvl_u
ocn_v = ts2.df.abs_vel_btm_v - ts2.df.rel_vel_dvl_v


bin01u = ts2.df.vel_bin0_beam0 - ts2.df.vel_bin1_beam0
bin01v = ts2.df.vel_bin0_beam1 - ts2.df.vel_bin1_beam1

bin12u = ts2.df.vel_bin1_beam0 - ts2.df.vel_bin2_beam0
bin12v = ts2.df.vel_bin1_beam1 - ts2.df.vel_bin2_beam1

bin23u = ts2.df.vel_bin2_beam0 - ts2.df.vel_bin3_beam0
bin23v = ts2.df.vel_bin2_beam1 - ts2.df.vel_bin3_beam1

bin34u = ts2.df.vel_bin3_beam0 - ts2.df.vel_bin4_beam0
bin34v = ts2.df.vel_bin3_beam1 - ts2.df.vel_bin4_beam1

roll_len=1

fig, ax = plt.subplots(figsize=(15,8))
sns.scatterplot(ts2.df.time, ocn_u.rolling(roll_len).median())
sns.scatterplot(ts2.df.time, ocn_v.rolling(roll_len).median())
# sns.scatterplot(ts2.df.time, bin01v.rolling(roll_len).median())
# sns.scatterplot(ts2.df.time, bin01u.rolling(roll_len).median())
# sns.scatterplot(ts2.df.time, bin12v.rolling(roll_len).median())
# sns.scatterplot(ts2.df.time, bin12u.rolling(roll_len).median())
sns.scatterplot(ts2.df.time, bin23v.rolling(roll_len).median())
sns.scatterplot(ts2.df.time, bin23u.rolling(roll_len).median())
plt.ylim(-0.25,0.25)



---
## Load or Parse Slocum Glider data 

In [ ]:
reload_modules()
directory = "/Users/zduguid/Dropbox (MIT)/MIT-WHOI/Kolumbo cruise 2019/zduguid/dbd-parsed/sentinel_2019-Nov/"
ts_flight_kolumbo_all = SlocumFlightController.SlocumFlightController.from_directory(directory, save=False, verbose=False)

In [ ]:
ts_flight_kolumbo_all.df

In [ ]:
ts_flight_kolumbo_all.df.to_csv("/Users/zduguid/Desktop/Kolumbo-glider-flight-computer.csv")

---
## Combine DVL data with Flight Controller and convert to UTM

In [ ]:
reload_modules()
SAVE = False
PLOT = True
glider      = "sentinel"
directory   = "/Users/zduguid/Dropbox (MIT)/MIT-WHOI/Kolumbo cruise 2019/zduguid/pd0-parsed/%s/" % glider
file_list   = [f for f in listdir(directory) if 
               isfile(join(directory,f)) and f.split('.')[-1] == 'CSV' and 
               len(f.split('_odometry')) != 2]

def get_utm_coords(m_lat, m_lon): 
    """TODO
    """
    SECS_IN_MIN = 60
    MIN_OFFSET = 100
    lat_min  = m_lat % MIN_OFFSET 
    lon_min  = m_lon % MIN_OFFSET 
    lat_dec  = (m_lat - lat_min)/MIN_OFFSET + lat_min/SECS_IN_MIN
    lon_dec  = (m_lon - lon_min)/MIN_OFFSET + lon_min/SECS_IN_MIN
    utm_pos  = utm.from_latlon(lat_dec, lon_dec)
    easting  = round(utm_pos[0],2)
    northing = round(utm_pos[1],2)
    zone     = utm_pos[2]
    return(easting, northing, zone)


# build dataframes for each odometry category 
dr_list  = []
odo_list = []
gps_list = []
wpt_list = []
all_list = []

# iterate through each dbd file and 
for f in file_list:
    SECS_IN_HOUR = 60*60
    SECS_IN_MIN  = 60
    df_pd0 = pd.read_csv(directory+f, header=0, index_col=0, parse_dates=True)
    start  = datetime.datetime.fromtimestamp(df_pd0.time[0])
    end    = datetime.datetime.fromtimestamp(df_pd0.time[-1])
    dur    = end - start 
    
    # sub-select a portion of glider flight computer variables
    df_dbd = ts_flight_kolumbo_all.df[str(start):str(end)].copy()
    print(f, "-------------------------")
    
    # extract first lat/lon gps position to use as origin
    for lat in df_dbd.m_gps_lat:
        if not np.isnan(lat):
            origin_m_lat = lat
            break
    origin_index = list(df_dbd.m_gps_lat).index(origin_m_lat)
    origin_m_lon = df_dbd.m_gps_lon[origin_index]
    origin_time  = datetime.datetime.fromtimestamp(df_dbd.time[origin_index])
    origin_hash  = df_dbd.m_filename_hash[origin_index]

    # convert to lat/lon coordinates to decimal fromat 
    origin_east,origin_north,origin_zone = get_utm_coords(origin_m_lat,origin_m_lon)
    
    # account for case when multiple dbd files span a single pd0 file
    print("    # dbd ensembles:  %d" % len(df_dbd))
    print("        # dbd files:  %d" % len(set(df_dbd.m_filename_hash)))
    print("         # missions:  %d" % len(set(df_dbd.m_mission_hash)))
    print("   input <lat, lon>: <%.2f, %.2f>" % (origin_m_lat, origin_m_lon))
    print("     UTM <  E,   N>: <%.2f, %.2f, zone=%d>\n\n" % (origin_east, origin_north, origin_zone))
    
    # extract positional offsets for the same moment in time
    sg2_x_offset = df_dbd[origin_time:].m_x_lmc[0]
    sg2_y_offset = df_dbd[origin_time:].m_y_lmc[0]
    dvl_x_offset = df_pd0[origin_time:].rel_pos_x[0]
    dvl_y_offset = df_pd0[origin_time:].rel_pos_y[0]

    # account for offsets in Glider and DVL file
    # TODO make this more SFB
    utm_dr_x  = df_dbd.m_x_lmc     - sg2_x_offset + origin_east
    utm_dr_y  = df_dbd.m_y_lmc     - sg2_y_offset + origin_north
    utm_dr_z  = df_dbd.m_depth 
    utm_gps_x = df_dbd.m_gps_x_lmc - sg2_x_offset + origin_east
    utm_gps_y = df_dbd.m_gps_y_lmc - sg2_y_offset + origin_north
    utm_wpt_x = df_dbd.c_wpt_x_lmc - sg2_x_offset + origin_east 
    utm_wpt_y = df_dbd.c_wpt_y_lmc - sg2_y_offset + origin_north 
    utm_odo_x = df_pd0.rel_pos_x   - dvl_x_offset + origin_east
    utm_odo_y = df_pd0.rel_pos_y   - dvl_y_offset + origin_north 
    utm_odo_z = df_pd0.rel_pos_z

    # collect data into indvidual DataFrames
    df_seg_dr = pd.DataFrame({
        'utm_dr_x' : utm_dr_x,
        'utm_dr_y' : utm_dr_y,
        'utm_dr_z' : utm_dr_z,
    })
    df_seg_gps = pd.DataFrame({
        'utm_gps_x' : utm_gps_x,
        'utm_gps_y' : utm_gps_y,
    })
    df_seg_odo = pd.DataFrame({
        'utm_odo_x' : utm_odo_x,
        'utm_odo_y' : utm_odo_y,
        'utm_odo_z' : utm_odo_z,
    })
    df_seg_wpt = pd.DataFrame({
        'utm_wpt_x' : utm_wpt_x,
        'utm_wpt_y' : utm_wpt_y,
    })
    
    # add dataframes to running list 
    dr_list.append(df_seg_dr)
    odo_list.append(df_seg_odo)
    gps_list.append(df_seg_gps)
    wpt_list.append(df_seg_wpt)

    # convert data to pandas DataFrame
    frames = {
        'utm_dr_x'  : utm_dr_x,
        'utm_dr_y'  : utm_dr_y,
        'utm_dr_z'  : utm_dr_z,
        'utm_gps_x' : utm_gps_x,
        'utm_gps_y' : utm_gps_y,
        'utm_wpt_x' : utm_wpt_x,
        'utm_wpt_y' : utm_wpt_y,
        'utm_odo_x' : utm_odo_x,
        'utm_odo_y' : utm_odo_y,
        'utm_odo_z' : utm_odo_z,    
    }
    df_all = pd.DataFrame(frames)
    all_list.append(df_all)
    
    # generate plots
    if PLOT:
        dvl_plotter.plot_odometry_and_dr(df_all, glider, '/Users/zduguid/Desktop/fig/%s.png' % f.split('.')[0])
    if SAVE:
        df_odo.to_csv('/Users/zduguid/Desktop/fig/%s.csv' % f.split('.')[0])

In [ ]:
# # COMBINE FILES 
# df_dr_all  = pd.concat(dr_list)
# df_odo_all = pd.concat(odo_list)
# df_gps_all = pd.concat(gps_list)
# df_wpt_all = pd.concat(wpt_list)
# df_all_all = pd.concat(all_list)

# # PLOTTING
# sns.scatterplot(df_dr_all.utm_dr_x, df_dr_all.utm_dr_y)
# sns.scatterplot(df_wpt_all.utm_wpt_x, df_wpt_all.utm_wpt_y)
# sns.scatterplot(df_gps_all.utm_gps_x, df_gps_all.utm_gps_y)
# sns.scatterplot(df_odo_all.utm_odo_x, df_odo_all.utm_odo_y)

# # SAVING
# df_dr_all.to_csv('/Users/zduguid/Downloads/dr_all.csv')
# df_odo_all.to_csv('/Users/zduguid/Downloads/df_odo_all.csv')
# df_gps_all.to_csv('/Users/zduguid/Downloads/df_gps_all.csv')
# df_wpt_all.to_csv('/Users/zduguid/Downloads/df_wpt_all.csv')
# df_all_all.to_csv('/Users/zduguid/Downloads/all_fields_all_dives.csv')

---
## Plotting

In [ ]:
reload_modules()
glider = "sentinel"
dvl_plotter.plot_profile(ts, glider)